# Tool Use

The `AgentChat` api provides a `ToolUseAssistantAgent` with presets for adding tools that the agent can call as part of it's response. 

:::{note}

The example presented here is a work in progress 🚧. Also, tool uses here assumed the `model_client` used by the agent supports tool calling. 
::: 

In [1]:
from autogen_agentchat.agents import ToolUseAssistantAgent
from autogen_agentchat.teams.group_chat import RoundRobinGroupChat
from autogen_core.components.models import OpenAIChatCompletionClient
from autogen_core.components.tools import FunctionTool

In AgentChat, a Tool is a function wrapped in the `FunctionTool` class exported from `autogen_core.components.tools`.  

In [2]:
async def get_weather(city: str) -> str:
    return f"The weather in {city} is 72 degrees and Sunny."

get_weather_tool = FunctionTool(get_weather, description="Get the weather for a city")

Finally, agents that use tools are defined in the following manner. 

-  An agent is instantiated based on the  `ToolUseAssistantAgent` class in AgentChat. The agent is aware of the tools it can use by passing a `tools_schema` attribute to the class, which is passed to the `model_client` when the agent generates a response.
-  An agent Team is defined that takes a list of `tools`.  Effectively, the `ToolUseAssistantAgent` can generate messages that call tools, and the team is responsible executing those tool calls and returning the results.

In [3]:
assistant = ToolUseAssistantAgent(
    "Weather_Assistant",
    model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
    tool_schema=[get_weather_tool.schema],
)
team = RoundRobinGroupChat([assistant], tools=[get_weather_tool])
result = await team.run("What's the weather in New York?")

# print(result)

--------------------------------------------------------------------------------
user:
What's the weather in New York?
--------------------------------------------------------------------------------
Weather_Assistant:
[FunctionCall(id='call_9Bh1HkKYVIQ2VAXuveQ5UMxp', arguments='{"city":"New York"}', name='get_weather')]
--------------------------------------------------------------------------------
tool_agent:
[FunctionExecutionResult(content='The weather in New York is 72 degrees and Sunny.', call_id='call_9Bh1HkKYVIQ2VAXuveQ5UMxp')]
--------------------------------------------------------------------------------
Weather_Assistant:
The weather in New York is 72 degrees and sunny. 

TERMINATE
